<a href="https://colab.research.google.com/github/fanny/information-retrieval/blob/master/lab3/tokenizing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import re

import pandas as pd
import numpy as np

import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import regexp_tokenize
from nltk.corpus import stopwords

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

data = pd.read_csv('https://raw.githubusercontent.com/fanny/information-retrieval/master/lab2/data/results.csv')
data.head(10)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,_id,author,date,section,sub_title,text,title,url
0,7,NaN,05/04/2019 |,agronegocio,NaN,NaN,\t\t\t\t,https://www.gazetadopovo.com.br/agronegocio/ex...
1,2,NaN,NaN,wiseup-news,NaN,NaN,NaN,https://www.gazetadopovo.com.br/wiseup-news/bo...
2,3,NaN,NaN,futebol,NaN,NaN,NaN,https://especiais.gazetadopovo.com.br/futebol/...
3,9,NaN,05/04/2019 |,agronegocio,NaN,NaN,Suspensão de frigoríficos não impediu alta de ...,https://www.gazetadopovo.com.br/agronegocio/ag...
4,1,The Daily Signal,NaN,ideias,NaN,Pessoas que buscam uma identidade transsexual ...,Populares,https://www.gazetadopovo.com.br/ideias/nao-se-...
5,5,Gazeta do Povo,NaN,curitiba,NaN,"Depois de uma semana de calor, Curitiba terá u...",Populares,https://www.gazetadopovo.com.br/curitiba/previ...
6,40,"Flávia Alves, especial para a Gazeta do Povo",NaN,viver-bem,NaN,As promessas são muitas: combater o envelhecim...,NaN,https://www.gazetadopovo.com.br/viver-bem/saud...
7,6,Agência Estado,NaN,republica,NaN,"O líder do MDB na Câmara, Baleia Rossi (SP), p...",Populares,https://www.gazetadopovo.com.br/republica/pec-...
8,20,Daniel Malucelli,[05/04/2019],esportes,mercado,O meia-atacante Maicosuel está com a situação ...,Paraná não descarta dispensa de Maicosuel ante...,https://www.gazetadopovo.com.br/esportes/paran...
9,10,Renyere Trovão,NaN,automoveis,NaN,Concretizar a compra de um carro novo nem semp...,Populares,https://www.gazetadopovo.com.br/automoveis/car...


In [0]:
clean_text = data['text'].dropna()
def filter_by_criteria(text, criteria):
  filtered_words = re.findall(criteria, text)
  filtered_words = ' '.join(filtered_words)
 
  return filtered_words

In [0]:
def get_words_filtered(criteria):
  filtered_text = clean_text.apply(lambda text: filter_by_criteria(text, criteria))
  all_filtered_text = filtered_text.str.cat(sep=' ')
  all_words_filtered = all_filtered_text.split(' ')

  all_words_filtered = [text for text in all_words_filtered if len(text) > 0]

  return set(all_words_filtered[:200])

## Tokenização

> Tokenização é o processo de formar palavras de uma sequência de caracteres em um documento  
**Obs: Texto extraido do livro**

E para formar essas palavras podemos usar alguns critérios, como:
- Quantidade de caracteres; Palavras hifenizadas; Caracteres especiais;
- Caracteres em Caixa Alta ou Baixa; Apostrófos; Números; Abreviações;

Cada critério desse, pode guardar uma importância, se tratando dessa base de dados de notícias, alguns poderão ser mais relevantes que outros e veremos o motivo.

Para conseguir concluir quais critérios usar no processo de tokenização, começaremos analisando os dados, garantindo assim que não ignoremos palavras que são importantes.

### Quantidade de caracteres

A primeira suposição é que palavras com quantidade menores que 3 geralmente não são importantes, porque representam artigos, preposições etc, que não agregam tanto valor na busca.

#### Resultados

In [17]:
get_words_filtered(r'\b\w{0,3}\b')


{'1',
 '500',
 '743',
 '744',
 '80',
 'A',
 'APA',
 'As',
 'Em',
 'Eu',
 'L',
 'Não',
 'O',
 'a',
 'aos',
 'as',
 'com',
 'da',
 'das',
 'de',
 'diz',
 'do',
 'e',
 'ela',
 'ele',
 'em',
 'eu',
 'fez',
 'gay',
 'há',
 'lo',
 'lê',
 'me',
 'na',
 'nem',
 'no',
 'nos',
 'não',
 'o',
 'os',
 'ou',
 'pai',
 'por',
 'que',
 'se',
 'ser',
 'seu',
 'si',
 'um',
 'uma',
 'uso',
 'à',
 'às',
 'é'}

A título de informação, só mostramos o set das 200 primeiras, mas olhando os dados podemos observar coisas interessantes como:

- Palavras capitalizadas indicam Siglas - APA, PIB, IPI, IVA, PEC, RJ
- Números só com um dígito, não são relevantes
- Ocorrem palavras com tamanho maior ou igual a dois que são importantes, indicam siglas, como 'RJ', 'PEC' e outras que o seu próprio significado tem muito valor, como 'gay' e 'pai'

Sendo assim, removeremos as palavras de tamanho 1 e as stopwords do brasil.


### Palavras hífenizadas

Provavelmente palavras com hífen serão muito importantes, porque elas representam dias da semana, nomes geográficos(Grã-Bretanha), certas espécies de animais e vegetais(Bem-te-vi, Couve-Flor) alguns adjetivos (mal-humorada) dentre outras.

In [18]:
get_words_filtered(r'[a-zA-Z]+\-[a-zA-Z]+')


{'Advocacia-Geral',
 'Centro-Oeste',
 'Controladoria-Geral',
 'DEM-RJ',
 'Deve-se',
 'Espera-se',
 'Estima-se',
 'Evidence-Based',
 'Jin-myung',
 'Junte-se',
 'MP-PR',
 'Mercedes-Benz',
 'Norte-Americana',
 'Norte-Sul',
 'PCdoB-AC',
 'PR-SP',
 'PRB-GO',
 'PSB-MG',
 'PSL-MG',
 'PSL-PR',
 'PSL-RN',
 'PSL-SP',
 'PSOL-RJ',
 'PT-MG',
 'PT-PR',
 'PT-RS',
 'PT-SP',
 'Procuradoria-Geral',
 'Recusar-se',
 'Rubro-Negro',
 'Santa-C',
 'Sul-Americana',
 'Trata-se',
 'a-feira',
 'a-tarefa',
 'advogado-geral',
 'ajudando-os',
 'anti-imperialistas',
 'anti-inflamat',
 'bate-boca',
 'batia-boca',
 'bem-estar',
 'bem-vinda',
 'bem-vindo',
 'boa-f',
 'busca-se',
 'caixas-preta',
 'contra-argumentar',
 'diga-se',
 'diretor-geral',
 'e-mail',
 'editor-chefe',
 'enviando-os',
 'es-p',
 'es-tanque',
 'ex-Iugosl',
 'ex-PPS',
 'ex-assessor',
 'ex-carpinteiro',
 'ex-comunistas',
 'ex-embaixador',
 'ex-goleiro',
 'ex-governador',
 'ex-ministro',
 'ex-presidente',
 'ex-presidentes',
 'ex-rep',
 'franco-brit',
 '

De fato, foram encontrados dias da semana, partidos, cargos, e muitas palavras importantes com hífen demonstrando que elas não devem ser removidas.

### Caracteres especiais

Também podem ser importantes, basta pensarmos em '%' que pode indicar percentuais, '$' que pode indicar dinheiro, 'º' que pode inficar uma posição, enfim, veremos a seguir, se esses símbolos são importantes no texto.

#### Resultados

In [53]:
get_words_filtered(r'(\w+\W+\w+)*')

['Pessoas que', '', 'buscam uma', '', 'identidade transsexual', '', 'não nasceram', '', 'assim e', '', 'crianças não', '', 'devem ser', '', 'estimuladas a', '', 'fazer a', '', '', 'transição” para', '', 'o sexo', '', 'oposto. Quem', '', 'diz é', '', 'uma obra', '', 'de referência', '', 'recomendada pela', '', 'Associação Norte', '', 'Americana de', '', 'Psicologia (APA', '', '', 'na sigla', '', 'em inglês', '', '', '', 'Ainda assim', '', '', 'todos os', '', 'dias ouço', '', 'algum pai', '', 'me dizer', '', 'que o', '', 'terapeuta de', '', 'seu filho', '', '', 'depois de', '', 'uma ou', '', 'duas consultas', '', '', 'recomendou enfaticamente', '', 'que o', '', 'pai permitisse', '', 'que o', '', 'filho mudasse', '', 'de nome', '', '', 'passasse a', '', 'usar outros', '', 'pronomes, vivesse', '', 'como se', '', 'fosse do', '', 'sexo oposto', '', 'e se', '', 'submetesse a', '', 'intervenções médicas', '', 'irreversíveis. Laura', '', 'Haynes, psicóloga', '', 'na Califórnia', '', '', 'recent

{'(APA',
 '1:',
 '744',
 'APA',
 'Ainda',
 'Americana',
 'Associação',
 'Califórnia',
 'Eles',
 'Entre',
 'Haynes,',
 'Laura',
 'Norte',
 'Pessoas',
 'Psicologia',
 'Quem',
 'a',
 'adolescente',
 'adulto',
 'afirmar',
 'afirmações',
 'algum',
 'alguém',
 'analisou',
 'antes',
 'assim',
 'assim,',
 'assunto.',
 'buscam',
 'ciência',
 'com',
 'como',
 'conclusiva',
 'consultas',
 'contrário',
 'criança,',
 'crianças',
 'da',
 'dando',
 'de',
 'defensores',
 'depois',
 'descobertas',
 'descobertas,',
 'destaque',
 'devem',
 'dias',
 'distorcidas',
 'diz',
 'dizem',
 'dizer',
 'do',
 'duas',
 'e',
 'em',
 'enfaticamente',
 'enganados',
 'essas',
 'estimuladas',
 'fazer',
 'filho',
 'fizer',
 'fosse',
 'gênero',
 'ideia',
 'identidade',
 'identificam',
 'inglês',
 'insistem',
 'intervenções',
 'irreversíveis.',
 'jamais',
 'lê',
 'manual',
 'me',
 'mudança',
 'mudarão',
 'mudasse',
 'médicas',
 'na',
 'nasceram',
 'nome',
 'não',
 'o',
 'obra',
 'oposto',
 'oposto.',
 'os',
 'ou',
 'outros'

Apesar de ser ruim visualizar, além da nossa suposição, foram encontrados certos delimitadores, que não acrescentam informações no texto como:
- '\r\n'
- '\xaO'  
Sendo assim, removeremos eles.

### Apóstrofos

Podem indicar marcas (Burguer’s King, McDonald’s) extrangeiras, ou uma “composição” de palavras como: pingo d’ água

#### Resultados


In [56]:
get_words_filtered(r"\w+\'\w*")

{"Amadurecimento'",
 "Gattaca'",
 "atualizar'",
 "chavismo'",
 "equipe'",
 "executivo'",
 "fontes'",
 "ganhar'",
 "iminente'",
 "merda'",
 "olavetes'",
 "papel'",
 "poços'",
 "revolução'",
 "unicórnio'",
 "vertical'"}

As palavras encontradas com apóstrofo,  provavlmente são erros de digitação/formatação, fazendo mais sentido da replace pra um caractere vazio, e não considerar o hífen na tokenização.

### Números

Números são extremamente importantes, podem indicar horas, salários, anos, posições, temperaturas, etc.

#### Resultados


In [58]:
get_words_filtered(r'\d+\w+')

{'00',
 '000',
 '004',
 '01',
 '03',
 '05',
 '07',
 '09',
 '10',
 '100',
 '11',
 '12',
 '13',
 '13º',
 '14',
 '15',
 '15ºC',
 '16',
 '18',
 '18ºC',
 '19',
 '1931',
 '1961',
 '1970',
 '1979',
 '1983',
 '1997',
 '1998',
 '1999',
 '19ºC',
 '1º',
 '20',
 '2002',
 '2004',
 '2005',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019',
 '2020',
 '21',
 '21ºC',
 '22',
 '22ºC',
 '23ºC',
 '24',
 '25',
 '25ºC',
 '260',
 '27',
 '27ºC',
 '28',
 '29',
 '297',
 '30',
 '300',
 '304',
 '32',
 '33',
 '349',
 '35',
 '36',
 '37',
 '377',
 '40',
 '41',
 '455',
 '457',
 '471',
 '500',
 '51',
 '53',
 '55',
 '553',
 '58',
 '5ª',
 '600',
 '623',
 '627',
 '630',
 '64',
 '669',
 '68',
 '682',
 '70',
 '738',
 '74',
 '743',
 '744',
 '750',
 '78',
 '7ª',
 '80',
 '801',
 '81',
 '818',
 '82',
 '839',
 '84',
 '841',
 '845',
 '852',
 '854',
 '858',
 '859',
 '860',
 '864',
 '870',
 '871',
 '879',
 '883',
 '90',
 '900',
 '91',
 '93',
 '94'}

Confirmando a suposição, não é interessante removê-los.

### Abreviações

Podem agregar muito valor pois servem para indicar cidades, siglas e etc..

### Resultados

In [59]:
get_words_filtered(r'(\w+\.\w+)*')

{'1.000',
 '1.008',
 '1.100',
 '1.250',
 '1.260',
 '1.297',
 '1.300',
 '1.542',
 '1.669',
 '1.695',
 '1.841',
 '10.ª',
 '105.714',
 '11.738',
 '12.846',
 '13.661',
 '13.ª',
 '16.635',
 '18.ª',
 '19.865',
 '2.143',
 '2.455',
 '2.457',
 '2.557',
 '2.565',
 '2.926',
 '2.º',
 '3.048',
 '3.530',
 '3.564',
 '3.813',
 '32.133',
 '32.200',
 '4.000',
 '4.004',
 '4.349',
 '4.377',
 '4.630',
 '4.756',
 '5.000',
 '5.553',
 '5.750',
 '5.ª',
 '6.000',
 '7.500',
 '76.200',
 '8.368',
 'Caracas.Especialistas',
 'Congresso.Nas',
 'Eletronuclear.O',
 'Globoesporte.com',
 'Paulo.O',
 'República.Segundo',
 'S.Paulo',
 'autoridades.O',
 'milhão.O'}

Assim, como no caso anterior, não é bom que eliminemos, pois temos números decimais, domínios de sites e etc.